In [1]:
import pandas as pd
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, cross_val_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import tree
import random
import itertools

%matplotlib inline

In [5]:
data = pd.read_csv('Data/Ready_for_ML/lnab_playoff.csv')
data.set_index(['Series_ID'],inplace=True)
data

,Year,Winner,Opponent 0,Homefield_advantage,Opponent 1,GF,PK SI,PK SI/GP,PK GA,PK GA/GP,...,10',GM,MP,PIM Total,PP OP,PP OP/GP,PP%,PPT,PP time/GP,Number of year
Series_ID,,,,,,,,,,,,,,,,,,,,,
0809_EV Zug Kloten Flyers,809,1,Kloten Flyers,1,EV Zug,3.0,-18.0,-0.36,-4.0,-0.08,...,11.0,-3.0,0.0,3.0,20.0,0.40,0.20,4586.0,92.0,0.0
0809_EV Zug SC Bern,809,1,EV Zug,0,SC Bern,-16.0,27.0,0.54,-5.0,-0.10,...,-2.0,0.0,-1.0,8.0,39.0,0.78,-5.01,4220.0,84.0,0.0
0809_Fribourg-Gottéron HC Davos,809,1,Fribourg-Gottéron,1,HC Davos,-30.0,-25.0,-0.50,11.0,0.22,...,4.0,-1.0,1.0,20.0,24.0,0.48,-3.74,3443.0,69.0,0.0
0809_Fribourg-Gottéron ZSC Lions,809,0,ZSC Lions,0,Fribourg-Gottéron,22.0,-31.0,-0.62,-17.0,-0.34,...,-4.0,0.0,-1.0,-134.0,-9.0,-0.18,4.81,-1737.0,-35.0,0.0
0809_Genève-Servette HC Kloten Flyers,809,0,Genève-Servette HC,1,Kloten Flyers,-17.0,26.0,0.52,0.0,0.00,...,0.0,0.0,2.0,120.0,-12.0,-0.24,-3.01,-3004.0,-60.0,0.0
0809_HC Davos HC Lugano,809,1,HC Davos,0,HC Lugano,9.0,14.0,0.28,-5.0,-0.10,...,-3.0,-1.0,-1.0,-97.0,-5.0,-0.10,-5.19,36.0,1.0,0.0
0809_HC Davos Kloten Flyers,809,1,Kloten Flyers,0,HC Davos,-10.0,-27.0,-0.54,-1.0,-0.02,...,8.0,1.0,0.0,65.0,45.0,0.90,-2.90,7321.0,146.0,0.0
0910_EHC Basel Sharks EHC Visp,910,0,EHC Basel Sharks,1,EHC Visp,-113.0,-10.0,-0.22,10.0,0.22,...,5.0,-3.0,-1.0,-67.0,-28.0,-0.62,-10.32,-1952.0,-43.0,0.0
0910_EHC Olten Lausanne HC,910,0,EHC Olten,0,Lausanne HC,21.0,-55.0,-1.22,-3.0,-0.06,...,-7.0,-3.0,0.0,-309.0,15.0,0.33,-2.17,3220.0,72.0,0.0


In [27]:
to_pred = data[data['Year'] == 1516]
to_train =  data[data['Year'] != 1516]

y_test = to_pred['Winner'].values
y_train = to_train['Winner'].values

X_train = to_train.drop(['Opponent 0', 'Winner', 'Year', 'Opponent 1'], axis=1)
X_test = to_pred.drop(['Opponent 0', 'Winner', 'Year', 'Opponent 1'], axis=1)

norm_X_train = MinMaxScaler().fit_transform(X_train)
norm_X_test = MinMaxScaler().fit_transform(X_test)

to_pred

,Year,Winner,Opponent 0,Homefield_advantage,Opponent 1,GF,PK SI,PK SI/GP,PK GA,PK GA/GP,...,10',GM,MP,PIM Total,PP OP,PP OP/GP,PP%,PPT,PP time/GP,Number of year
Series_ID,,,,,,,,,,,,,,,,,,,,,
1516_EHC Olten EHC Visp,1516,1,EHC Visp,1,EHC Olten,-14.0,3.0,0.07,10.0,0.22,...,3.0,-1.0,0.0,37.0,-13.0,-0.29,-2.72,-1423.0,-32.0,0.0
1516_EHC Olten HC Ajoie,1516,1,HC Ajoie,0,EHC Olten,-7.0,-28.0,-0.62,-1.0,-0.03,...,-1.0,-3.0,0.0,-134.0,23.0,0.51,2.73,2718.0,60.0,0.0
1516_EV Zug HC Lugano,1516,0,EV Zug,0,HC Lugano,4.0,9.0,0.18,1.0,0.02,...,3.0,0.0,0.0,28.0,6.0,0.12,5.65,598.0,12.0,0.0
1516_Fribourg-Gottéron Genève-Servette HC,1516,0,Genève-Servette HC,0,Fribourg-Gottéron,12.0,42.0,0.84,-6.0,-0.12,...,0.0,-1.0,0.0,31.0,-22.0,-0.44,4.84,-1069.0,-21.0,0.0
1516_Genève-Servette HC HC Lugano,1516,1,Genève-Servette HC,1,HC Lugano,3.0,28.0,0.56,2.0,0.04,...,4.0,-1.0,0.0,42.0,-14.0,-0.28,8.74,-1173.0,-23.0,0.0
1516_HC Ajoie HC La Chaux-de-Fonds,1516,0,HC Ajoie,0,HC La Chaux-de-Fonds,14.0,-31.0,-0.69,-15.0,-0.34,...,1.0,-3.0,-1.0,-117.0,19.0,0.42,4.30,1698.0,38.0,0.0
1516_HC Ajoie SC Rapperswil-Jona Lakers,1516,1,HC Ajoie,1,SC Rapperswil-Jona Lakers,9.0,-8.0,-0.17,-2.0,-0.05,...,-1.0,-6.0,-1.0,-206.0,14.0,0.31,-0.66,2044.0,45.0,0.0
1516_HC Davos Kloten Flyers,1516,1,HC Davos,0,Kloten Flyers,27.0,22.0,0.44,-13.0,-0.26,...,7.0,5.0,0.0,238.0,17.0,0.34,-3.26,934.0,19.0,0.0
1516_HC Davos SC Bern,1516,1,SC Bern,1,HC Davos,-29.0,0.0,0.00,11.0,0.22,...,-3.0,-3.0,0.0,-58.0,36.0,0.72,5.31,2058.0,41.0,0.0


In [132]:
model = MLPClassifier(hidden_layer_sizes=(50,), early_stopping=False, activation='logistic', max_iter=1000, alpha=0.001, solver='lbfgs')

model.fit(norm_X_train, y_train)

y_pred = model.predict(norm_X_test)

print(roc_auc_score(y_test, y_pred))
print(y_test)
print(y_pred)

0.475
[1 1 0 0 1 0 1 1 1 0 1 1 1 1]
[0 1 0 1 1 1 1 1 0 1 0 1 1 1]
